In [5]:
from kalman.kalman_functions import *
import pandas as pd
import cv2
from helper_function.VizLibRerun import *

In [6]:
import rerun as rr

In [1]:

labels_file_path = r'C:\Users\Hasan\OneDrive\Desktop\Projects\pfas_finalproject\data\reect\seq_01\labels.txt'
images_folder_path = r'C:\Users\Hasan\OneDrive\Desktop\Projects\pfas_finalproject\data\reect\seq_01\image_02\data'
depth_maps_path = r'C:\Users\Hasan\OneDrive\Desktop\Projects\pfas_finalproject\data\sgbm_depth_maps\seq_01'


# Read the labels.txt file
headers = [
    "frame", "track id", "type", "truncated", "occluded", "alpha",
    "bbox_left", "bbox_top", "bbox_right", "bbox_bottom",
    "dimensions_height", "dimensions_width", "dimensions_length",
    "location_x", "location_y", "location_z",
    "rotation_y", "score"
]
df = pd.read_csv(labels_file_path, sep=' ', names=headers)

# Define constants
dt = 0.1036  # Time difference between frames
depth = 0  # Assumed constant depth for this case
max_distance = 300  # 100 for Sequence 02, 100 for Sequence 01, 300 for Sequence 03
max_distance_3d = 0.5  # Test
max_missing_frames = 40  # 40 for Sequence 02, 40 for Sequence 01

# Initialize variables
# missing_ids = []          # IDs of objects missing in current frame
# tracked_ids = []          # IDs of objects being tracked
# new_ids = []              # IDs of new detections in current frame
locations_dict = {}       # Stores bounding boxes for each track_id
sizes_dict = {}           # Stores width and height for each track_id
previous_frame_ids = []   # IDs detected in previous frame
previous_frame_locations = {}  # Bounding boxes from previous frame
frame_counter = 1         # Counter for the current frame
reassociation_map = {}    # Maps new_ids to missing_ids
kalman_filters = {}       # Kalman filter instances for each track_id
missing_counts = {}       # Counts how long each missing_id has been missing

# 3D kalman Filters Variables         
kalman_filters_3d = {}    # Kalman filter instances for each track_id in 3D
locations_dict_3d = {}    # Stores 3D bounding boxes for each track_id
previous_frame_locations_3d = {}  # 3D Bounding boxes from previous frame
missing_counts_3d = {}    # Counts how long each missing_id has been missing in 3D
previous_frame_ids_3d = []  # IDs detected in previous frame in 3D

path_hist = {}
path_hist_kalman = {}


# Get sorted list of frames

P_rect_02 = np.array([
    [7.070493e+02, 0.000000e+00, 6.040814e+02, 4.575831e+01],
    [0.000000e+00, 7.070493e+02, 1.805066e+02, -3.454157e-01],
    [0.000000e+00, 0.000000e+00, 1.000000e+00, 4.981016e-03]
])

K = P_rect_02[:3, :3]
unique_frames = sorted(df["frame"].unique()) 

NameError: name 'pd' is not defined

In [ ]:
# Main processing loop
from os import path
from helper_function.bbhelp import extract_image_info


rr.init("Tracking Demo",spawn=True)
# rr.notebook_show(width=1750, height=800)
for i,frame in enumerate(unique_frames):
    try:
        rr.set_time_sequence("Frame ID: ", i)
        rr.set_time_seconds('Frame ID: ', i)
        print(f"Processing frame {frame}")
        # Clear update tracking lists
        updated_ids = []
        predicted_only_ids = []
        about_to_remove_ids = []

        updated_ids_3d = []
        predicted_only_ids_3d = []
        about_to_remove_ids_3d = []

        # Step 2: Retrieve Frame Data
        current_frame_ids, current_frame_locations,current_frame_locations_3d = get_frame_data(frame)
        labels = extract_image_info(i,labels_file=labels_file_path)

        for obj in labels:
            if obj['track_id'] not in path_hist:
                path_hist[obj['track_id']] = [[obj['location'][0],obj['location'][2],-obj['location'][1]]]
            else:
                path_hist[obj['track_id']].append([obj['location'][0],obj['location'][2],-obj['location'][1]])

        for track_id ,path in path_hist.items():
            if len(path) > 2:
                rr.log(f"/world/path{track_id}", rr.LineStrips3D(path,colors=(0,255,0)))

        rr.log(f"/world/obj_gt{obj['track_id']}", objs2boxs(labels,color=(0,255,0)))
   
        preds = labels.copy()
        print(f"Current frame IDs: {current_frame_ids}")

        # Read the corresponding image
        image_path = os.path.join(images_folder_path, f"{frame:006d}.png")
        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            continue



        image = cv2.imread(image_path)
        depth = np.load(os.path.join(depth_maps_path, f"{frame:06d}.npy"))

        for pred in preds:
            if depth[int(pred['center'][1]), int(pred['center'][0])] != 0 and pred['occluded'] == 0:
                pred_pos = depth[int(pred['center'][1]), int(pred['center'][0])] * np.linalg.inv(K) @ np.array([pred['center'][0], pred['center'][1], 1]).reshape(-1, 1)
                pred['location'] = pred_pos.flatten().tolist()
                # rr.log(f"/world/obj_pred{pred['track_id']}", objs2boxs([pred]))
            else:
                preds.remove(pred)

        current_frame_ids_3d = [pred['track_id'] for pred in preds]
        current_frame_locations_3d = {pred['track_id']: pred['location'] for pred in preds}

        
        # Verify if the image has been loaded correctly
        if image is None or image.size == 0:
            print(f"Error loading image: {image_path}")
            continue

        frame_height, frame_width = image.shape[:2]

        # Step 3: Determine States (track_states)
        missing_ids, tracked_ids, new_ids = track_states(current_frame_ids, previous_frame_ids)
        missing_ids_3d, tracked_ids_3d, new_ids_3d = track_states(current_frame_ids_3d, previous_frame_ids_3d)
        print(f"Missing IDs: {missing_ids}")
        print(f"Tracked IDs: {tracked_ids}")
        print(f"New IDs: {new_ids}")

        # Update missing_counts for missing_ids
        # 2D
        for id_ in missing_ids:
            missing_counts[id_] = missing_counts.get(id_, 0) + 1
        # 3D
        for id_ in missing_ids_3d:
            missing_counts_3d[id_] = missing_counts_3d.get(id_, 0) + 1

        # Step 4: Process missing_ids
        missing_ids_prediction = {}
        for missing_id in missing_ids.copy():
            if missing_id in kalman_filters:
                # Predict the position
                kalman_filter = kalman_filters[missing_id]
                kalman_filter.predict()
                predicted_center = kalman_filter.get_current_state()[:2, 0]
                # Reconstruct the bbox using the last known size
                size = sizes_dict.get(missing_id, (0, 0))  # Get width and height
                predicted_bbox = extract_bbox_from_state(kalman_filter.get_current_state(), size)
                missing_ids_prediction[missing_id] = predicted_center  # Store predicted center
                # Update locations_dict
                locations_dict[missing_id] = predicted_bbox
                predicted_only_ids.append(missing_id)
            else:
                print(f"Kalman filter not found for missing ID {missing_id}")
                missing_ids.remove(missing_id)
            
        # 3D
        missing_ids_prediction_3d = {}
        for missing_id in missing_ids_3d.copy():
            if missing_id in kalman_filters_3d:
                # Predict the position
                kalman_filter = kalman_filters_3d[missing_id]
                kalman_filter.predict()
                pose = kalman_filter.get_current_state()[:3, 0]
                missing_ids_prediction_3d[missing_id] = pose  # Store predicted center
                locations_dict_3d[missing_id] = pose
                predicted_only_ids_3d.append(missing_id)
            else:
                print(f"Kalman filter not found for missing ID {missing_id}")
                missing_ids_3d.remove(missing_id)


        # Remove predicted objects that are out of frame
        for missing_id in missing_ids.copy():
            if missing_id in kalman_filters:
                kalman_filter = kalman_filters[missing_id]
                predicted_center = kalman_filter.get_current_state()[:2, 0]
                
                # Check if predicted center is out of frame
                if (predicted_center[0] < 0 or predicted_center[0] > frame_width or 
                    predicted_center[1] < 0 or predicted_center[1] > frame_height):
                    # Remove from tracking
                    print(f"Removed Missing ID {missing_id} because predicted center is out of frame")
                    missing_ids.remove(missing_id)
                    del kalman_filters[missing_id]
                    del missing_counts[missing_id]
                    locations_dict.pop(missing_id, None)
                    sizes_dict.pop(missing_id, None)

        # 3D
        # for missing_id in missing_ids_3d.copy():
        #     if missing_id in kalman_filters_3d:
        #         kalman_filter = kalman_filters_3d[missing_id]
        #         pose = kalman_filter.get_current_state()[:3, 0]
        #         # Check if predicted center is out of frame
        #         if pose[0] < 0 or pose[0] > frame_width or pose[1] < 0 or pose[1] > frame_height:
        #             # Remove from tracking
        #             print(f"Removed Missing ID {missing_id} because predicted center is out of frame")
        #             missing_ids_3d.remove(missing_id)
        #             del kalman_filters_3d[missing_id]
        #             del missing_counts_3d[missing_id]
        #             locations_dict_3d.pop(missing_id, None)

        # Removing predicted objects that are out of frame not needed for 3D tracking

        # Step 5: Match new_ids to missing_ids_prediction and Initialize Kalman Filters for Unmatched new_ids
        matched_ids, reassociation_map, unmatched_new_ids = match_new_ids_to_missing_predictions(
            new_ids, missing_ids_prediction.copy(), current_frame_locations, max_distance
        )
        matched_ids_3d, reassociation_map_3d, unmatched_new_ids_3d = match_new_ids_to_missing_predictions_3d(
            new_ids_3d, missing_ids_prediction_3d.copy(), current_frame_locations_3d, max_distance_3d
        )
        print(f"Matched IDs: {matched_ids}")
        print(f"Unmatched New IDs: {unmatched_new_ids}")

        # Initialize Kalman filters for unmatched new_ids
        for new_id in unmatched_new_ids:
            bbox = current_frame_locations[new_id]
            center = calculate_center(bbox)
            measurement = [center[0], center[1]]
            kalman_filter = KalmanFilter(initial_state=measurement)
            kalman_filters[new_id] = kalman_filter
            sizes_dict[new_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])  # Store width and height
            if new_id not in tracked_ids:
                tracked_ids.append(new_id)
            locations_dict[new_id] = bbox
            updated_ids.append(new_id)  # Newly initialized, considered updated

        # 3D
        for new_id in unmatched_new_ids_3d:
            pose = current_frame_locations_3d[new_id]
            kalman_filter = KalmanFilter3D(initial_state=pose)
            kalman_filters_3d[new_id] = kalman_filter
            if new_id not in tracked_ids_3d:
                tracked_ids_3d.append(new_id)
            locations_dict_3d[new_id] = pose
            updated_ids_3d.append(new_id)

        # Remove matched missing_ids from missing_ids and missing_counts
        for missing_id, new_id in matched_ids:
            if missing_id in missing_ids:
                missing_ids.remove(missing_id)
            if missing_id in missing_counts:
                del missing_counts[missing_id]
            # Update Kalman filter mapping if necessary
            if missing_id in kalman_filters:
                pass
            else:
                kalman_filters[missing_id] = kalman_filters.pop(new_id)
            # Update sizes_dict
            sizes_dict[missing_id] = sizes_dict.get(missing_id, sizes_dict.get(new_id))
            if new_id in sizes_dict:
                del sizes_dict[new_id]
            # Update locations_dict
            locations_dict[missing_id] = current_frame_locations[new_id]
            if new_id in locations_dict:
                del locations_dict[new_id]

        # 3D
        for missing_id, new_id in matched_ids_3d:
            if missing_id in missing_ids_3d:
                missing_ids_3d.remove(missing_id)
            if missing_id in missing_counts_3d:
                del missing_counts_3d[missing_id]
            # Update Kalman filter mapping if necessary
            if missing_id in kalman_filters_3d:
                pass
            else:
                kalman_filters_3d[missing_id] = kalman_filters_3d.pop(new_id)
            # Update locations_dict
            locations_dict_3d[missing_id] = current_frame_locations_3d[new_id]
            if new_id in locations_dict_3d:
                del locations_dict_3d[new_id]

        # Step 6: Update Kalman Filter for matched_ids
        for missing_id, new_id in matched_ids:
            # Get Kalman filter for missing_id (ensure it exists)
            kalman_filter = kalman_filters[missing_id]
            # Predict step
            kalman_filter.predict()
            # Update step with new measurements
            bbox = current_frame_locations[new_id]
            center = calculate_center(bbox)
            measurement = [center[0], center[1]]
            kalman_filter.update(measurement)
            # Update sizes_dict with the new size
            sizes_dict[missing_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])
            # Update locations_dict with the updated bbox
            updated_bbox = extract_bbox_from_state(kalman_filter.get_current_state(), sizes_dict[missing_id])
            locations_dict[missing_id] = updated_bbox
            # Replace new_id with missing_id in tracked_ids
            if new_id in tracked_ids:
                tracked_ids.remove(new_id)
            if missing_id not in tracked_ids:
                tracked_ids.append(missing_id)
            updated_ids.append(missing_id)  # Mark as updated with measurement

        # 3D
        for missing_id, new_id in matched_ids_3d:
            # Get Kalman filter for missing_id (ensure it exists)
            kalman_filter = kalman_filters_3d[missing_id]
            # Predict step
            kalman_filter.predict()
            # Update step with new measurements
            pose = current_frame_locations_3d[new_id]
            kalman_filter.update(pose)
            # Update locations_dict with the updated pose
            locations_dict_3d[missing_id] = pose
            # Replace new_id with missing_id in tracked_ids
            if new_id in tracked_ids_3d:
                tracked_ids_3d.remove(new_id)
            if missing_id not in tracked_ids_3d:
                tracked_ids_3d.append(missing_id)
            updated_ids_3d.append(missing_id)

        # Step 7: Process tracked_ids
        matched_missing_ids = [m[0] for m in matched_ids]
        for tracked_id in tracked_ids.copy():
            if tracked_id in matched_missing_ids:
                continue  # Already updated in Step 6

            # Ensure Kalman filter exists for tracked_id
            if tracked_id not in kalman_filters:
                if tracked_id in current_frame_locations:
                    bbox = current_frame_locations[tracked_id]
                else:
                    bbox = locations_dict.get(tracked_id, [0, 0, 0, 0])  # Default bbox if not available
                center = calculate_center(bbox)
                measurement = [center[0], center[1]]
                kalman_filter = KalmanFilter(initial_state=measurement)
                kalman_filters[tracked_id] = kalman_filter
                sizes_dict[tracked_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])
                print(f"Initialized Kalman filter for tracked ID {tracked_id}")

            kalman_filter = kalman_filters[tracked_id]
            # Predict step
            kalman_filter.predict()

            if tracked_id in current_frame_locations:
                # Update step with measurement
                bbox = current_frame_locations[tracked_id]
                center = calculate_center(bbox)
                measurement = [center[0], center[1]]
                kalman_filter.update(measurement)
                # Update sizes_dict with the new size
                sizes_dict[tracked_id] = (bbox[2] - bbox[0], bbox[3] - bbox[1])
                # Update locations_dict with the updated bbox
                updated_bbox = extract_bbox_from_state(kalman_filter.get_current_state(), sizes_dict[tracked_id])
                locations_dict[tracked_id] = updated_bbox
                updated_ids.append(tracked_id)  # Mark as updated
            else:
                # Handle predictions for tracked IDs not in current frame
                predicted_center = kalman_filter.get_current_state()[:2, 0]
                # Reconstruct the bbox using the last known size
                size = sizes_dict.get(tracked_id, (0, 0))
                predicted_bbox = extract_bbox_from_state(kalman_filter.get_current_state(), size)
                locations_dict[tracked_id] = predicted_bbox
                predicted_only_ids.append(tracked_id)

        # 3D
        matched_missing_ids_3d = [m[0] for m in matched_ids_3d]
        for tracked_id in tracked_ids_3d.copy():
            if tracked_id in matched_missing_ids_3d:
                continue

            # Ensure Kalman filter exists for tracked_id
            if tracked_id not in kalman_filters_3d:
                if tracked_id in current_frame_locations_3d:
                    pose = current_frame_locations_3d[tracked_id]
                else:
                    pose = locations_dict_3d.get(tracked_id, [0, 0, 0])
                kalman_filter = KalmanFilter3D(initial_state=pose)
                kalman_filters_3d[tracked_id] = kalman_filter
                print(f"Initialized Kalman filter for tracked ID {tracked_id}")

            kalman_filter = kalman_filters_3d[tracked_id]
            # Predict step
            kalman_filter.predict()

            if tracked_id in current_frame_locations_3d:
                # Update step with measurement
                pose = current_frame_locations_3d[tracked_id]
                kalman_filter.update(pose)
                # Update locations_dict with the updated pose
                locations_dict_3d[tracked_id] = pose
                updated_ids_3d.append(tracked_id)

            else:
                # Handle predictions for tracked IDs not in current frame
                pose = kalman_filter.get_current_state()[:3, 0]
                locations_dict_3d[tracked_id] = pose
                predicted_only_ids_3d.append(tracked_id)

        # Step 8: Update Tracking States for Next Frame
        # 2D
        previous_frame_ids = tracked_ids + missing_ids
        previous_frame_locations = {id_: locations_dict[id_] for id_ in previous_frame_ids if id_ in locations_dict}
        # 3D
        previous_frame_ids_3d = tracked_ids_3d + missing_ids_3d
        previous_frame_locations_3d = {id_: locations_dict_3d[id_] for id_ in previous_frame_ids_3d if id_ in locations_dict_3d}

        # Remove stale missing_ids
        for missing_id in missing_ids.copy():
            if missing_counts[missing_id] > max_missing_frames:
                # Remove object from tracking
                print(f"Removed Missing ID {missing_id} after exceeding max missing frames")
                missing_ids.remove(missing_id)
                del kalman_filters[missing_id]
                del missing_counts[missing_id]
                locations_dict.pop(missing_id, None)
                sizes_dict.pop(missing_id, None)

        # 3D
        for missing_id in missing_ids_3d.copy():
            if missing_counts_3d[missing_id] > max_missing_frames:
                # Remove object from tracking
                print(f"Removed Missing ID {missing_id} after exceeding max missing frames")
                missing_ids_3d.remove(missing_id)
                del kalman_filters_3d[missing_id]
                del missing_counts_3d[missing_id]
                locations_dict_3d.pop(missing_id, None)
        # Increment frame_counter
        frame_counter += 1

        # Visualization
        # Define a mapping of object types to colors for classification
        classification_colors = {
            "Pedestrian": (34, 139, 34),  # Green
            "Car": (0, 0, 128),         # Navy Blue
            "Cyclist": (139, 0, 0),     # Dark Red
            "misc": (128, 0, 128)       # Purple (for unclassified objects)
        }

        # print(f'Location 3D: {locations_dict_3d}')
        # Kalman Tracked 3D objects
        tracked_objs = labels.copy()

        for pred in tracked_objs:
            if pred['track_id'] in locations_dict_3d.keys():
                pred['location'] = locations_dict_3d[pred['track_id']]
                rr.log(f"/world/obj_kalman{pred['track_id']}", objs2boxs([pred],color=(0,0,255)))
            else:
                tracked_objs.remove(pred)

        for obj in tracked_objs:
            if obj['track_id'] not in path_hist_kalman:
                path_hist_kalman[obj['track_id']] = [[obj['location'][0],obj['location'][2],-obj['location'][1]]]
            else:
                path_hist_kalman[obj['track_id']].append([obj['location'][0],obj['location'][2],-obj['location'][1]])

        for track_id ,path in path_hist_kalman.items():
            if len(path) > 2:
                rr.log(f"/world/tracked_path{track_id}", rr.LineStrips3D(path,colors=(0,0,255)))
        # Draw bounding boxes
        for track_id, bbox in locations_dict.items():
            x_top_left, y_top_left, x_bottom_right, y_bottom_right = bbox

            # Get the object type for the current track_id
            # Replace with actual classification lookup logic as needed
            object_type = df[df['track id'] == track_id]['type'].iloc[0] if not df[df['track id'] == track_id].empty else "misc"
            color = classification_colors.get(object_type, (128, 128, 128))  # Default to gray if type is unknown

            # Draw the bounding box with the classification color
            cv2.rectangle(image, (int(x_top_left), int(y_top_left)), (int(x_bottom_right), int(y_bottom_right)), color, 2)

            # Draw text label with background rectangle
            text = f"ID: {track_id}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            thickness = 1
            text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
            text_width, text_height = text_size
            # Draw rectangle behind text
            cv2.rectangle(image, (int(x_top_left), int(y_top_left) - text_height - 5),
                        (int(x_top_left) + text_width, int(y_top_left)), color, -1)
            # Draw text
            cv2.putText(image, text, (int(x_top_left), int(y_top_left) - 5), font, font_scale, (255, 255, 255), thickness)

            # Draw the predicted center point
            kalman_filter = kalman_filters[track_id]
            predicted_center = kalman_filter.get_current_state()[:2, 0]

            # Draw predicted center (blue circle)
            cv2.circle(image, (int(predicted_center[0]), int(predicted_center[1])), 5, (255, 0, 0), -1)  # Blue for predicted center

            # If updated with measurement, draw measurement point (yellow circle)
            if kalman_filter.updated_with_measurement:
                measurement_center = kalman_filter.x[:2, 0]
                cv2.circle(image, (int(measurement_center[0]), int(measurement_center[1])), 5, (0, 255, 255), -1)  # Yellow for measurement

        # Add frame number to the top-left corner with a background
        frame_text = f"Frame: {frame}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        text_color = (255, 255, 255)  # White text
        bg_color = (0, 0, 0)  # Black background

        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(frame_text, font, font_scale, font_thickness)

        # Set position for text and background
        text_x = 10
        text_y = 30
        bg_x1 = text_x - 5
        bg_y1 = text_y - text_height - 5
        bg_x2 = text_x + text_width + 5
        bg_y2 = text_y + baseline

        # Draw the background rectangle
        cv2.rectangle(image, (bg_x1, bg_y1), (bg_x2, bg_y2), bg_color, -1)

        # Put the text on top of the background
        cv2.putText(image, frame_text, (text_x, text_y), font, font_scale, text_color, font_thickness)

        # Show the image with the bounding boxes
        # cv2.imshow("Image", image)
        # key = cv2.waitKey(100) & 0xFF
        # if key == ord('q'):
        #     break


        # for pred in preds:
        #     rr.log(f"/world/obj_pred{pred['track_id']}", obj2box(pred))
        rr.log("/world/image_02", rr.Image(image[:,:,::-1]))
        rr.log("/world/depth", rr.Image(depth))


        # rr.log("/world/pred_depth", rr.Image(midas_pred))
        # rr.log("/world/image", rr.Image(img[:,:,::-1]))
        # rr.log('/world/pixel_mask', rr.Image(pixels_mask.astype(np.uint8)*255))

    except Exception as e:
        print(f"Exception occurred at frame {frame}: {e}")
        traceback.print_exc()
        break

# cv2.destroyAllWindows()

Processing frame 0
Current frame IDs: [0, 1, 5, 6, 7, 8, 9]
Missing IDs: []
Tracked IDs: []
New IDs: [0, 1, 5, 6, 7, 8, 9]
Matched IDs: []
Unmatched New IDs: [0, 1, 5, 6, 7, 8, 9]
Processing frame 1
Current frame IDs: [0, 1, 5, 6, 7, 8, 9]
Missing IDs: []
Tracked IDs: [0, 1, 5, 6, 7, 8, 9]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 2
Current frame IDs: [0, 1, 5, 6, 7, 8, 9]
Missing IDs: []
Tracked IDs: [0, 1, 5, 6, 7, 8, 9]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 3
Current frame IDs: [0, 1, 5, 6, 7, 8, 9]
Missing IDs: []
Tracked IDs: [0, 1, 5, 6, 7, 8, 9]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 4
Current frame IDs: [0, 1, 5, 6, 7, 8, 9]
Missing IDs: []
Tracked IDs: [0, 1, 5, 6, 7, 8, 9]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing frame 5
Current frame IDs: [0, 1, 5, 6, 7, 8, 9]
Missing IDs: []
Tracked IDs: [0, 1, 5, 6, 7, 8, 9]
New IDs: []
Matched IDs: []
Unmatched New IDs: []
Processing fram